In [70]:
import numpy as np
from PIL import Image
import cv2
import pickle

In [71]:
import os

try:
    BASE_DIR = os.path.dirname(os.path.abspath(__file__))
except NameError:
    import sys
    BASE_DIR = os.path.dirname(os.path.abspath(sys.argv[0]))

In [72]:
BASE_DIR = 'C:\\Users\\itadmin'

In [73]:
image_dir = os.path.join(BASE_DIR, "images")

In [74]:
image_dir 

'C:\\Users\\itadmin\\images'

In [75]:
face_cascade = cv2.CascadeClassifier('cascades/data/haarcascade_frontalface_alt2.xml')
recognizer = cv2.face.LBPHFaceRecognizer_create()

In [80]:
y_labels = []
x_train = []
current_id = 0
label_ids = {}

for root, dirs, files in os.walk(image_dir):
    for file in files:
        if file.endswith("png") or file.endswith("jpg"):
            path = os.path.join(root, file)
            label = os.path.basename(os.path.dirname(path)).replace(" ","-").lower()
            #or label = os.path.basename(root).replace(" ","-").lower()
            #print(label, path)
            
            if not label in label_ids:
                label_ids[label] = current_id
                current_id += 1
                
            id_ = label_ids[label]
            #print(label_ids)
            
            #y_labels.append(label) #we want some number
            #x_train.append(path) #verify this image and turn it into a numpy array, also to gray
            
            pil_image = Image.open(path).convert("L") # .convert ('L') turns it into  gray
            size = (550,550)
            final_image = pil_image.resize(size, Image.ANTIALIAS)
            image_array = np.array(final_image, "uint8") # every pixel turning into numpy array
            #print(image_array)
            faces = face_cascade.detectMultiScale(image_array, scaleFactor = 1.5, minNeighbors = 5)
            
            for (x,y,w,h) in faces:
                roi = image_array[y:y+h, x:x+w]
                x_train.append(roi)
                y_labels.append(id_)
            
print(y_labels)
print(x_train)

[0, 0, 1, 1, 1, 1, 1, 1]
[array([[ 55,  58,  58, ...,  23,  27,  29],
       [ 70,  64,  83, ...,  24,  17,  17],
       [ 73,  81,  85, ...,  18,  23,  23],
       ...,
       [ 62,  68,  69, ..., 100,  84,  89],
       [ 63,  67,  67, ..., 108,  97,  78],
       [ 61,  57,  53, ..., 103, 102,  93]], dtype=uint8), array([[235, 237, 241, ..., 203, 194, 197],
       [237, 234, 237, ..., 204, 194, 194],
       [238, 234, 233, ..., 203, 196, 193],
       ...,
       [231, 234, 236, ..., 206, 204, 199],
       [231, 234, 235, ..., 206, 205, 201],
       [230, 233, 235, ..., 208, 201, 202]], dtype=uint8), array([[254, 254, 254, ..., 254, 254, 254],
       [254, 254, 254, ..., 254, 254, 254],
       [254, 254, 254, ..., 254, 254, 254],
       ...,
       [247, 254, 255, ...,  96,  96,  98],
       [247, 205, 155, ...,  88,  95, 101],
       [139, 111, 101, ...,  91,  97, 103]], dtype=uint8), array([[ 31,  30,  21, ...,  86, 121, 111],
       [ 27,  33,  19, ...,  67, 132, 114],
       [ 22, 

In [77]:
path

'C:\\Users\\itadmin\\images\\sherin\\6.png'

In [78]:
with open("labels.pickle", 'wb') as f: # writing bytes as file
    pickle.dump(label_ids, f)

recognizer.train(x_train, np.array(y_labels)) #training numpy arrays
recognizer.save("trainner.yml")